In [1]:
# Import necessary modules
import sys
import os

# Ensure the current directory (root folder) is in the Python module path
sys.path.append(os.getcwd())

import torch
from maze_dataset.generation import LatticeMazeGenerators
from maze_dataset.dataset.rasterized import MazeDatasetConfig, MazeDataset, RasterizedMazeDataset

In [2]:
def get_device():
    """ Use CUDA or MPS if available, otherwise use CPU """
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

In [3]:
def get_mazes(maze_size=9, num_mazes=10, gen='dfs_perc', percolation=0.0, deadend_start=True):
    """ Generate mazes of the given size and number, 
        from the given dataset, and load to device"""
    
    assert maze_size % 2 == 1
    grid_n = (maze_size + 1) // 2
    
    # Generate base maze dataset
    if gen == 'dfs':
        maze_ctor = LatticeMazeGenerators.gen_dfs
        maze_ctor_kwargs = dict()
    elif gen == 'dfs_perc':
        maze_ctor = LatticeMazeGenerators.gen_dfs_percolation
        maze_ctor_kwargs = dict(p=percolation)
    elif gen == 'percolation':
        maze_ctor = LatticeMazeGenerators.gen_percolation
        maze_ctor_kwargs = dict(p=percolation)
    endpoint_kwargs=dict(deadend_start=deadend_start, 
                         endpoints_not_equal=True,
                         except_on_no_valid_endpoint=False)

    base_dataset = MazeDataset.from_config(
        MazeDatasetConfig(
            name='test',
            grid_n=grid_n,
            n_mazes=num_mazes,
            seed=42,
            maze_ctor=maze_ctor, 
            maze_ctor_kwargs=maze_ctor_kwargs,
            endpoint_kwargs=endpoint_kwargs
        ),
        gen_parallel = True,
        local_base_path='data/maze_dataset/',
    )

    # Print information about base_dataset object
    cfg = base_dataset.cfg
    mazes = base_dataset.mazes
    print(f'{cfg = }')
    print(f'{len(mazes) = }')
    print(f"{['M' if m is not None else 'N' for m in mazes]}")

    # Generate rasterized maze dataset
    dataset = RasterizedMazeDataset.from_base_MazeDataset(
        base_dataset=base_dataset,
        added_params=dict(
            remove_isolated_cells=True,
            extend_pixels=True, # maps from 1x1 to 2x2 pixels and adds 3 padding
        )
    )

    dataset = dataset.get_batch(idxs=None)

    # Get inputs
    inputs = dataset[0,:,:,:]
    inputs = inputs / 255.0
    inputs = inputs.permute(0, 3, 1, 2)
    inputs = inputs.float().detach().to(get_device(), dtype=torch.float32)

    # Get solutions
    solutions = dataset[1,:,:, :]
    solutions = solutions / 255.0
    solutions = solutions.permute(0, 3, 1, 2)
    solutions, _ = torch.max(solutions, dim=1)
    solutions = solutions.float().detach().to(get_device(), dtype=torch.float32) 

    return inputs, solutions

In [4]:
# Create percolated maze

inputs, solutions = get_mazes(
    maze_size=9, 
    num_mazes=10,
    percolation=0.5,
    deadend_start=True)

cfg = MazeDatasetConfig(name='test', seq_len_min=1, seq_len_max=512, seed=42, applied_filters=[], grid_n=5, n_mazes=10, maze_ctor=<function LatticeMazeGenerators.gen_dfs_percolation at 0x000001E513B11360>, maze_ctor_kwargs={'p': 0.5}, endpoint_kwargs={'deadend_start': True, 'endpoints_not_equal': True, 'except_on_no_valid_endpoint': False})
len(mazes) = 10
['M', 'M', 'N', 'M', 'N', 'M', 'N', 'M', 'M', 'N']


AttributeError: 'NoneType' object has no attribute 'as_pixels'